 # 4. Generate an animation with PyVista
 This notebook is only intended for users who want to explore more about the possibilities offered by the joint use of `toughio` and `pyvista`.
 It shows how to animate the evolution of the CO<sub>2</sub> plume with the geology in the background.

 Time series output can be more easily visualized in ParaView after executing the command line script `toughio-export`:

 ```
 toughio-export OUTPUT_ELEME.csv -m Preprocessing/mesh.pickle -f xdmf
 ```

 First, we import `toughio` and `pyvista`.

In [1]:
import toughio
import pyvista

pyvista.set_plot_theme("document")


 Then, we unpickle the mesh file and import all the time steps in the output file (`OUTPUT_ELEME.csv` in TOUGH3).

 Note that we reorder the output data according to the order of labels in the mesh since output data are not necessarily exported by TOUGH in the same order as they were defined (especially when using the parallel version).

In [2]:
mesh = toughio.read_mesh("../Preprocessing/mesh.pickle")
output = toughio.read_output("../OUTPUT_ELEME.csv", labels_order=mesh.labels)


 Now, we convert our unpickled `toughio.Mesh` object to a `pyvista.UnstructuredGrid`.

In [3]:
mesh = mesh.to_pyvista()


 Let's define some filter layers:
 - The bottom layer will be the geology (with the boundary elements removed),
 - The top layer will be the mesh wireframe.


 In between, we will display the CO<sub>2</sub> plume.

In [4]:
mesh.set_active_scalars("material")
material = mesh.threshold((0.0, 5.0))
wire = material.extract_all_edges()


 To create an animation, we simply loop over the different time steps in the output data, create a static figure (as we would to visualize a single time step) and save a screenshot as an image array. CO<sub>2</sub> plume can be extracted by removing the cells with a gas saturation value lower than a threshold value (here 0.1).

 Note that cell data are converted to point data to get a smoother plume.

In [5]:
frames = []
for i, out in enumerate(output):
    for k, v in out.data.items():
        mesh.cell_arrays[k] = v
    mesh_in = mesh.cell_data_to_point_data()

    # Saturation
    mesh_in.set_active_scalars("SAT_G")
    saturation = mesh_in.threshold((0.1, 1.0))

    # Plot
    p = pyvista.Plotter(window_size=(800, 800), notebook=True)
    p.add_mesh(
        material,
        scalars="material",
        cmap="YlGnBu",
        show_scalar_bar=False,
    )
    p.add_mesh(
        saturation,
        stitle="Gas saturation",
        cmap="viridis_r",
        clim=(0.0, 0.5),
        n_colors=20,
        edge_color=(0.5, 0.5, 0.5),
        scalar_bar_args={
            "height": 0.1,
            "width": 0.5,
            "position_x": 0.75,
            "position_y": 0.01,
            "vertical": False,
            "n_labels": 6,
            "fmt": "%.1f",
            "title_font_size": 20,
            "font_family": "arial",
            "shadow": True,
        },
    )
    p.add_mesh(
        wire,
        color="black",
        opacity=0.25,
    )
    title = p.add_text(
        "Time: {:.2f} years".format(out.time / 3600.0 / 24.0 / 365.25),
        position="upper_right",
        font_size=12,
        shadow=True,
    )
    p.camera_position = [
        (1000.0, -4500.0, -1550.0),
        (1000.0, 0.5, -1550.0),
        (0.0, 0.0, 1.0),
    ]
    if i == len(output)-1:
        p.screenshot("sample.png", transparent_background=True)
    p.close()

    frames.append(p.image)


 Finally, we can export our animation using `imageio` (which is a dependency of `pyvista`).

In [6]:
import imageio

imageio.mimsave("sample.gif", frames, duration=0.1)
